In [9]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import pandas as pd
import numpy as np
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
import gc
tqdm.pandas()
gc.collect()

36

# LOAD

In [10]:
%%time
df = pd.read_feather('../data/2021玉山人工智慧公開挑戰賽冬季賽訓練資料集.feather')
df = df.loc[df.dt>=12,:] #取最近一年即可
print(df.shape,df.isnull().sum().sum())
df.head()

(19786604, 53) 1509444
Wall time: 1min 9s


,dt,chid,shop_tag,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,domestic_offline_amt_pct,...,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
13188928,12.0,10115966.0,15,1.0,9678.450445,1.0,0.0,0.0,0.0,1.00,...,1.0,4.0,15.0,1.0,1.0,30.0,133000.939458,0.0,5.0,1.0
13188938,12.0,10115966.0,18,1.0,9624.614033,1.0,0.0,0.0,0.0,1.00,...,1.0,4.0,15.0,1.0,1.0,30.0,133000.939458,0.0,5.0,1.0
13188944,12.0,10115966.0,25,1.0,2338.181448,1.0,0.0,0.0,0.0,1.00,...,1.0,4.0,15.0,1.0,1.0,30.0,133000.939458,0.0,5.0,1.0
13188946,12.0,10115966.0,26,2.0,16627.150110,2.0,0.0,0.0,0.0,1.00,...,1.0,4.0,15.0,1.0,1.0,30.0,133000.939458,0.0,5.0,1.0
13188957,12.0,10115966.0,37,13.0,18485.113446,3.0,10.0,0.0,0.0,0.27,...,1.0,4.0,15.0,1.0,1.0,30.0,133000.939458,0.0,5.0,1.0


# 處理缺失值

In [11]:
%%time
# Function to calculate missing values by column# Funct 
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns
miss_table = missing_values_table(df)
miss_table

Your selected dataframe has 53 columns.
There are 9 columns that have missing values.
Wall time: 6.5 s


,Missing Values,% of Total Values
slam,1023527,5.2
gender_code,242749,1.2
age,242749,1.2
trdtp,365,0.0
educd,14,0.0
masts,10,0.0
naty,10,0.0
poscd,10,0.0
cuorg,10,0.0


In [12]:
mean_fill_col = ['slam'] # 數值型用均值補值
cat_fill_col = ['gender_code','age','trdtp','educd','masts','naty','poscd','cuorg'] # 類別型用一個-999補值
for i in tqdm(mean_fill_col):
    df[i] = df[i].fillna(df[i].mean())
for i in tqdm(cat_fill_col):
    df[i] = df[i].fillna(-999)
miss_table = missing_values_table(df)
miss_table

100%|██████████| 8/8 [00:01<00:00,  6.55it/s]


Your selected dataframe has 53 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


#  以下根據chid做groupby操作,並且因應各種不同類型的欄位作客製化統計運算(沿時間維度取mean之類的)

In [13]:
%%time
def create_groupby_chid_df(df):
    
    # 1 groupby('chid')根據時間維度取最後一筆
    final_col = ['age','primary_card','trdtp','educd','gender_code','masts','poscd','naty','cuorg'] 
    var1 = df.groupby('chid')[final_col].agg(lambda x:x.values[-1]) # 最後一筆 
    
    # 2 groupby('chid')根據時間維度算平均百分比
    mean_col = df.columns[df.columns.str.contains('pct')]
    var2 = df.groupby('chid')[mean_col].agg('mean')
    
    # 3 groupby('chid')根據時間維度算norm_cnt(月平均消費次數(cnt取mean),再接一個normalize操作)
    card_cnt_col = df.columns[df.columns.str.contains('card')&df.columns.str.contains('cnt')]
    var3 = df.groupby('chid')[card_cnt_col].agg('mean')
    var3.loc[:,:] = normalize(var3)
    
    # 4 groupby('chid')根據時間維度算norm_cnt(月平均消費次數(cnt取mean),再接一個normalize操作)
    domestic_overseas_col = df.columns[(df.columns.str.contains('domestic')|df.columns.str.contains('overseas'))&(df.columns.str.contains('cnt'))]
    var4 = df.groupby('chid')[domestic_overseas_col].agg('mean')
    var4.loc[:,:] = normalize(var4)

    # 5 groupby('chid')根據時間維度算平均值(slam(正卡信用額度平均值),txn_amt(消費金額平均值),txn_cnt(消費次數平均值))
    other_mean_col = ['slam','txn_amt','txn_cnt']
    var5 = df.groupby('chid')[other_mean_col].agg('mean')

    # 1 2 3 4 5 concat 起來
    df_groupby_chid = pd.concat([var1,var2,var3,var4,var5],axis=1).reset_index()
    
    print(set(df.columns) - set(df_groupby_chid.columns))

    return df_groupby_chid

print(df.shape)
df_groupby_chid = create_groupby_chid_df(df)
df_groupby_chid

(19786604, 53)
{'shop_tag', 'dt'}
Wall time: 48.8 s


,chid,age,primary_card,trdtp,educd,gender_code,masts,poscd,naty,cuorg,...,card_13_txn_cnt,card_14_txn_cnt,card_other_txn_cnt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,slam,txn_amt,txn_cnt
0,10000000.0,4.0,1.0,2.0,2.0,1.0,2.0,3.0,1.0,30.0,...,0.0,0.0,1.000000,0.433816,0.901002,0.0,0.000000,79557.742574,9701.914585,1.600000
1,10000001.0,5.0,1.0,11.0,4.0,0.0,2.0,3.0,1.0,30.0,...,0.0,0.0,0.000000,0.969334,0.245747,0.0,0.000000,62730.806297,7977.560539,1.816327
2,10000002.0,3.0,1.0,13.0,4.0,0.0,2.0,99.0,1.0,30.0,...,0.0,0.0,0.000000,0.894427,0.447214,0.0,0.000000,79291.240239,6647.343469,1.000000
3,10000003.0,6.0,1.0,15.0,6.0,0.0,2.0,99.0,1.0,30.0,...,0.0,0.0,1.000000,1.000000,0.000000,0.0,0.000000,73926.209680,8288.806894,1.500000
4,10000004.0,2.0,1.0,15.0,5.0,1.0,2.0,99.0,1.0,30.0,...,0.0,0.0,0.000000,0.666667,0.666667,0.0,0.333333,48209.277501,7076.586473,1.111111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,10499995.0,4.0,1.0,5.0,3.0,1.0,1.0,99.0,1.0,30.0,...,0.0,0.0,1.000000,0.000000,0.000000,0.0,1.000000,87397.101503,3716.334885,1.000000
499996,10499996.0,3.0,1.0,11.0,3.0,1.0,2.0,2.0,1.0,30.0,...,0.0,0.0,0.000000,0.972387,0.233373,0.0,0.000000,55463.131267,11491.514477,1.291667
499997,10499997.0,3.0,1.0,8.0,6.0,0.0,1.0,2.0,1.0,30.0,...,0.0,0.0,0.000000,0.976187,0.216930,0.0,0.000000,72218.713500,6568.010877,1.692308
499998,10499998.0,4.0,1.0,13.0,6.0,1.0,2.0,99.0,1.0,30.0,...,0.0,0.0,0.000000,0.768980,0.000000,0.0,0.639273,62417.960432,13384.207376,3.304348


In [14]:
df_groupby_chid['chid'].nunique() #確認應該有50萬筆

500000

# 做資料預處理(one_hot和標準化轉換),把資料欄位都變成數值型態特徵

In [15]:
%%time
from sklearn.preprocessing import scale
import warnings 
warnings.filterwarnings('ignore')

def preprocess_for_knn(df_groupby_chid):
    # 對類別欄位做one hot encoding
    categorical_df = pd.DataFrame()
    categorical_features = ['masts','educd','trdtp','naty','poscd','cuorg','gender_code','age','primary_card']
    for c_name in categorical_features:
        df_groupby_chid[c_name] = pd.to_numeric(df_groupby_chid[c_name])
    for c_name in categorical_features:
        one_hot = pd.get_dummies(df_groupby_chid[c_name])
        one_hot.columns = [ c_name + '_' + str(i) for i in one_hot.columns]
        if len(categorical_df)==0:
            categorical_df = one_hot
        else:
            categorical_df = pd.concat([categorical_df,one_hot],axis=1)
    df_groupby_chid = df_groupby_chid.drop(categorical_features,axis=1) # 刪掉原始類別欄位
    df_groupby_chid = pd.concat([df_groupby_chid,categorical_df],axis=1) # 加入one hot版本類別欄位
    
    # 做標準化轉換
    scale_col = set(df_groupby_chid.columns) - set(['chid'])
    scale_col = list(scale_col)
    df_groupby_chid[scale_col] = scale(df_groupby_chid[scale_col])
    
    return df_groupby_chid

df_groupby_chid_preprocessed = preprocess_for_knn(df_groupby_chid)
df_groupby_chid_preprocessed


Wall time: 14.2 s


,chid,domestic_offline_amt_pct,domestic_online_amt_pct,overseas_offline_amt_pct,overseas_online_amt_pct,card_1_txn_amt_pct,card_2_txn_amt_pct,card_3_txn_amt_pct,card_4_txn_amt_pct,card_5_txn_amt_pct,...,age_2.0,age_3.0,age_4.0,age_5.0,age_6.0,age_7.0,age_8.0,age_9.0,primary_card_0.0,primary_card_1.0
0,10000000.0,-0.496861,0.773526,-0.096182,-0.435581,-0.437361,-0.432504,-0.164106,-0.633559,-0.193868,...,-0.419145,-0.621392,1.680625,-0.476502,-0.315847,-0.136068,-0.042275,-0.007616,-0.29984,0.29984
1,10000001.0,0.367163,-0.192007,-0.096182,-0.435581,0.425479,1.644711,-0.164106,-0.633559,-0.193868,...,-0.419145,-0.621392,-0.595017,2.098627,-0.315847,-0.136068,-0.042275,-0.007616,-0.29984,0.29984
2,10000002.0,-0.000899,0.219296,-0.096182,-0.435581,-0.437361,-0.432504,-0.164106,1.747482,-0.193868,...,-0.419145,1.609290,-0.595017,-0.476502,-0.315847,-0.136068,-0.042275,-0.007616,-0.29984,0.29984
3,10000003.0,1.126289,-1.040320,-0.096182,-0.435581,-0.437361,-0.432504,-0.164106,-0.633559,-0.193868,...,-0.419145,-0.621392,-0.595017,-0.476502,3.166093,-0.136068,-0.042275,-0.007616,-0.29984,0.29984
4,10000004.0,-0.902649,0.639167,-0.096182,0.797201,-0.437361,-0.432504,-0.164106,1.747482,-0.193868,...,2.385812,-0.621392,-0.595017,-0.476502,-0.315847,-0.136068,-0.042275,-0.007616,-0.29984,0.29984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,10499995.0,-2.255274,-1.040320,-0.096182,7.489447,-0.437361,-0.432504,-0.164106,-0.633559,-0.193868,...,-0.419145,-0.621392,1.680625,-0.476502,-0.315847,-0.136068,-0.042275,-0.007616,-0.29984,0.29984
499996,10499996.0,0.562695,-0.410512,-0.096182,-0.435581,-0.437361,2.450882,-0.164106,-0.633559,-0.193868,...,-0.419145,1.609290,-0.595017,-0.476502,-0.315847,-0.136068,-0.042275,-0.007616,-0.29984,0.29984
499997,10499997.0,0.416161,-0.246762,-0.096182,-0.435581,2.648710,-0.432504,-0.164106,-0.633559,-0.193868,...,-0.419145,1.609290,-0.595017,-0.476502,-0.315847,-0.136068,-0.042275,-0.007616,-0.29984,0.29984
499998,10499998.0,0.758728,-1.040320,-0.096182,0.425835,-0.437361,-0.432504,-0.164106,-0.633559,-0.193868,...,-0.419145,-0.621392,1.680625,-0.476502,-0.315847,-0.136068,-0.042275,-0.007616,-0.29984,0.29984


# 特徵維度太多先用pca降維

In [16]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95)
X = df_groupby_chid_preprocessed.drop('chid',axis=1).values
X_pca = pca.fit_transform(X)
X_pca.shape

(500000, 103)

# 建立knn模型

In [17]:
%%time
from sklearn.neighbors import NearestNeighbors
k = 42
nbrs = NearestNeighbors(
    n_neighbors=k, 
    algorithm='ball_tree',
    leaf_size=300,
    n_jobs=-1,
    p=2).fit(X_pca)
nbrs

Wall time: 20.1 s


NearestNeighbors(algorithm='ball_tree', leaf_size=300, n_jobs=-1,
                 n_neighbors=42)

# 試用knn模型

In [18]:
distances, indices = nbrs.kneighbors(X_pca[[0]]) # 將該chid的"特徵"丟入"knn"
source_chid = df_groupby_chid_preprocessed.iloc[0]['chid']
target_chids = df_groupby_chid_preprocessed.loc[indices[0][-(k-1):]]['chid'].values.tolist()
print('distance由小到大',distances[0][1:])
print(source_chid,'==>',target_chids)

distance由小到大 [0.38917685 0.45809351 0.56302992 0.88129723 1.85003472 2.03992116
 2.1019519  2.13330417 2.39982295 2.44751426 2.70380089 2.70970619
 2.75486607 3.00574448 3.02358015 3.10399237 3.16458216 3.31701738
 3.3742335  3.40240162 3.42296305 3.4398845  3.44076945 3.44391936
 3.44559154 3.48085929 3.57619226 3.59657695 3.63678512 3.65955386
 3.6611803  3.66684972 3.68206051 3.68562302 3.7043036  3.71385767
 3.72639276 3.74467888 3.76653088 3.77872787 3.79052611]
10000000.0 ==> [10160475.0, 10141830.0, 10030875.0, 10054131.0, 10095545.0, 10396783.0, 10054232.0, 10467301.0, 10409524.0, 10161173.0, 10478866.0, 10377725.0, 10073873.0, 10109205.0, 10258960.0, 10107286.0, 10077169.0, 10220812.0, 10080426.0, 10484428.0, 10133405.0, 10125650.0, 10142145.0, 10003724.0, 10379110.0, 10387106.0, 10413369.0, 10253544.0, 10202863.0, 10060414.0, 10479321.0, 10202350.0, 10198916.0, 10450140.0, 10015760.0, 10014180.0, 10077132.0, 10195476.0, 10463006.0, 10442187.0, 10413235.0]


In [19]:
source_chid_shop_tag = df.loc[df.chid == source_chid,'shop_tag'].value_counts()[:3].index.values.tolist()
target_chid_shop_tag = df.loc[df.chid.isin(target_chids),'shop_tag'].value_counts()[:3].index.values.tolist()
print(source_chid_shop_tag,target_chid_shop_tag)
print('交集百分比:',len(set(source_chid_shop_tag)&set(target_chid_shop_tag))/3)

['10', '45', '12'] ['other', '45', '10']
交集百分比: 0.6666666666666666


# 以上實驗驗證了相似的chid有相似的shop_tag
# 因此未來在預測上如果當下chid無法集滿三個shop_tag,可以搜索相似chid的shop_tag

In [20]:
import joblib

# 保存knn模型
joblib.dump(nbrs,'../model/nbrs.pkl')
# 保存X_pca
joblib.dump(X_pca,'../model/X_pca_for_knn.pkl')
# 保存df_groupby_chid_preprocessed
df_groupby_chid_preprocessed.to_feather('../data/df_groupby_chid_preprocessed.feather')

In [21]:
#ok